In [120]:
import pdfplumber
import re
import pandas as pd
import numpy as np

In [27]:
with pdfplumber.open('receipt.pdf') as pdf:
    # iterate over each page
    page = pdf.pages[0]
    tables = page.extract_tables()
    for table in tables:
        print(table)
        print('*'*100)


[['', ''], [None, ''], ['Olivia delivered your order\nYour order from ALDI was placed on February 2nd, 2023 and delivered on February 2nd,\n2023 at 6:50 PM\nItems Found Adjustment\n14 1\nA D J U S T M E N T S ( A L D I ) 1\nN O T C H A R G E D\nFriendly Farms Nonfat Black Cherry on the Bottom Greek Yogurt\n$0.00\n(5.3 oz)\nRefunded 5\nI T E M S F O U N D ( A L D I )\n1 4\nC A N N E D G O O D S\nSimply Nature Organic Low Sodium Chicken Broth\n(32 oz)\n$2.09\n1 x $2.09\nDeutsche Küche Hearty Pea Soup\n(28 oz)\n$3.09\n1 x $3.09\nSimply Nature Organic Vegetable Broth\n(32 oz)\n$2.09\n1 x $2.09\nD A I R Y & E G G S\nFriendly Farms Nonfat Blueberry on the Bottom Greek Yogurt\n(5.3\n$3.25\noz)\n5 x $0.65\nFriendly Farms Whole Milk\n(0.5 gal)\n$1.95\n1 x $1.95\nFriendly Farms Whole Milk\n(1 gal)\n$3.25\n1 x $3.25\nM E AT & S E A F O O D\nKirkwood Fresh Boneless Skinless Chicken Thighs\n(~ 1.82 lb)\n$5.86\n1.78 lb x $3.29\nP R O D U C E\nAvocado\n(each)\n$2.20\n4 x $0.55\nRoma Tomato Package\n(

In [105]:
reg_expressions = {} #mandatory features of items
# reg_expressions['item_cost'] = r'\)\n\ (.*?) \n'
reg_expressions['item_cost'] = r'\n\$\d+\.\d+\n'#r'\n$\d+\.\d+\n'
reg_expressions['item_cost_description'] = r'\d+ x \$\d+\.\d+'
reg_expressions['item_name'] = r'^[A-Z,\s,a-z].*'

optional_reg_expressions = {} #optional fratures of items
optional_reg_expressions['item_quantity'] = r'\((.*?)\)'
for dicts in [reg_expressions, optional_reg_expressions]:
    for key, value in dicts.items():
        dicts[key] = re.compile(value)

string = 'Simply Nature Organic Vegetable Broth\n(32 oz)\n$2.09\n1 x $2.09'

Items = {}
all_keys = list(reg_expressions.keys()) + list(optional_reg_expressions.keys())
for key in all_keys:
    Items[key] = []
status = [0]*len(reg_expressions.keys())
new_item = {}
for ind, (key, value) in enumerate(reg_expressions.items()):
        match = value.findall(string)
        if match:
            status[ind] = 1
            new_item[key] = re.sub(r'\n','', match[0])
            # print(key, new_item[key])
if sum(status) == len(reg_expressions.keys()): #have successfully identified the item line
    for ind, (key, value) in enumerate(optional_reg_expressions.items()):
        match = value.findall(string)
        if match:
            new_item[key] = re.sub(r'\n','', match[0])
            # print(key, new_item[key])
        else:
            new_item[key] = ''
    for key in all_keys:
        print(key, new_item[key])
        Items[key].append(new_item[key])

Items = pd.DataFrame(Items)
Items.to_csv('Items.txt', sep='\t')

item_cost $2.09
item_cost_description 1 x $2.09
item_name Simply Nature Organic Vegetable Broth
item_quantity 32 oz
{'item_cost': ['$2.09'], 'item_cost_description': ['1 x $2.09'], 'item_name': ['Simply Nature Organic Vegetable Broth'], 'item_quantity': ['32 oz']}


In [65]:
print(reg_expressions.keys())
print(optional_reg_expressions.keys())

dict_keys(['item_cost', 'item_description', 'item_name'])
dict_keys(['item_quantity'])


In [110]:
table = tables[-1]
print(table)
# with pdfplumber.open('receipt.pdf') as pdf:
#     for page in pdf.pages:
#         text = page.extract_text()
#         print(text)

[['C A N N E D G O O D S'], ['Simply Nature Organic Low Sodium Chicken Broth\n(32 oz)\n$2.09\n1 x $2.09'], ['Deutsche Küche Hearty Pea Soup\n(28 oz)\n$3.09\n1 x $3.09'], ['Simply Nature Organic Vegetable Broth\n(32 oz)\n$2.09\n1 x $2.09'], ['D A I R Y & E G G S'], ['Friendly Farms Nonfat Blueberry on the Bottom Greek Yogurt\n(5.3\n$3.25\noz)\n5 x $0.65'], ['Friendly Farms Whole Milk\n(0.5 gal)\n$1.95\n1 x $1.95'], ['Friendly Farms Whole Milk\n(1 gal)\n$3.25\n1 x $3.25'], ['M E AT & S E A F O O D'], ['Kirkwood Fresh Boneless Skinless Chicken Thighs\n(~ 1.82 lb)\n$5.86\n1.78 lb x $3.29'], ['P R O D U C E'], ['Avocado\n(each)\n$2.20\n4 x $0.55'], ['Roma Tomato Package\n(1 lb)\n$5.16\n4 x $1.29'], ['Strawberries, Package\n(1 lb)\n$3.29\n1 x $3.29'], ['Bananas\n(~ 0.42 lb)\n$1.61\n3.09 lb x $0.52'], ['Cilantro\n(1 ct)\n$1.70\n2 x $0.85'], ['Yellow Potato Bag\n(each)\n$4.39\n1 x $4.39'], ['Red Onion Bag\n(2 lb bag)\n$2.99\n1 x $2.99'], ['']]


In [129]:
def read_pdf(infile):    
    #intializing features
    reg_expressions = {} #mandatory features of items
    # reg_expressions['item_cost'] = r'\)\n\ (.*?) \n'
    reg_expressions['item_cost'] = r'\n\$\d+\.\d+\n'#r'\n$\d+\.\d+\n'
    reg_expressions['item_cost_description'] = r'\d+ x \$\d+\.\d+'
    reg_expressions['item_name'] = r'^[A-Z,\s,a-z].*'

    optional_reg_expressions = {} #optional fratures of items
    optional_reg_expressions['item_quantity'] = r'\((.*?)\)'
    for dicts in [reg_expressions, optional_reg_expressions]:
        for key, value in dicts.items():
            dicts[key] = re.compile(value)

    Items = {}
    all_keys = list(reg_expressions.keys()) + list(optional_reg_expressions.keys())
    for key in all_keys:
            Items[key] = []

            
    # reading the pdf        
    with pdfplumber.open(infile) as pdf:
    # iterate over each page
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables[1:]:
                for [string] in table:    
                    status = [0]*len(reg_expressions.keys())
                    new_item = {}
                    for ind, (key, value) in enumerate(reg_expressions.items()):
                            match = value.findall(string)
                            if match:
                                status[ind] = 1
                                new_item[key] = re.sub(r'\n','', match[0])
                                # print(key, new_item[key])
                    if sum(status) == len(reg_expressions.keys()): #have successfully identified the item line
                        for ind, (key, value) in enumerate(optional_reg_expressions.items()):
                            match = value.findall(string)
                            if match:
                                new_item[key] = re.sub(r'\n','', match[0])
                                # print(key, new_item[key])
                            else:
                                new_item[key] = ''
                        for key in all_keys:
                            Items[key].append(new_item[key])

    Items = pd.DataFrame(Items)
    Items = Items[['item_cost_description', 'item_quantity', 'item_name', 'item_cost']]
    Items['item_cost'] = Items['item_cost'].map(lambda x:float(x.replace('$','')))
    return Items
    
infile = 'receipt.pdf'
Items = read_pdf(infile)  
Items.to_csv('Items.txt', sep='\t')
Items.head()

,item_cost_description,item_quantity,item_name,item_cost
0,1 x $2.09,32 oz,Simply Nature Organic Low Sodium Chicken Broth,2.09
1,1 x $3.09,28 oz,Deutsche Küche Hearty Pea Soup,3.09
2,1 x $2.09,32 oz,Simply Nature Organic Vegetable Broth,2.09
3,5 x $0.65,,Friendly Farms Nonfat Blueberry on the Bottom ...,3.25
4,1 x $1.95,0.5 gal,Friendly Farms Whole Milk,1.95


In [127]:
Items.keys()

Index(['item_cost', 'item_cost_description', 'item_name', 'item_quantity'], dtype='object')